In [11]:
import sched, time
import requests
import json
import os
from requests.exceptions import HTTPError
import pandas as pd

In [38]:
sequence_dir = "/media/nas/PeopleDetection/up_realsense_frames/Data_Seq/AVI"
sequence_records = "/media/nas/PeopleDetection/up_realsense_frames/Data_Seq/sequences_labeling_record (copy).csv"
server_url = "http://visionclassificationtool.inov.pt"
sequence_labels = '/media/nas/PeopleDetection/up_realsense_frames/Data_Seq/labels'
max_uploads = 10
min_size = 50 * 1024
interval = 2 * 60 * 60

Plan is:
Every 30 minutes:
- Read sequence_labeling.json
- - for each sequence where !sent
- - - upload to classification app
- - - - if !error
- - - - - set sequence.sent to True
- - - write to sequence_labeling.json

In [64]:
def login():
    r = requests.post(server_url+'/api/users/login', json={
        'email': 'ricardo.coelho@inov.pt',
        'password': 'inov'
    })
    r.raise_for_status()
    return r.json()["token"]

def upload_sequence(sequence, session):
    filename = sequence_dir+'/'+sequence['name']+'.mp4'
    if os.path.getsize(filename) < min_size:
        return True
    try:
        dataset_res = session.post(server_url+'/api/dataset', json={
            'name': sequence['name']
        })
        dataset_res.raise_for_status()
        dataset = dataset_res.json()
        
        files = {}
        files[dataset['_id']+'_RGB'] = (filename, open(filename, 'rb'), 'video/mp4')
        
        upload_res = session.post(server_url+'/api/upload/dataset', files=files)
        upload_res.raise_for_status()
        
        job_res = session.post(server_url+'/api/job', json={
            'dataset': dataset['_id'],
            'classes': ["Person"],
            'type': 'Tracking',
            'priority': 1,
            'channel': 'RGB'
        })
        job_res.raise_for_status()
        return True
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return False

def upload_new_sequences(scheduler): 
    print("Uploading sequences...")
    try:
        token = login()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return
    session = requests.Session()
    session.headers.update({"Authorization": 'Bearer '+token})
    sequences = pd.read_csv(sequence_records)
    try:
        jobs_res = session.get(server_url+'/api/job')
        jobs_res.raise_for_status()
        jobs = jobs_res.json()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return
    active_jobs = [j for j in jobs if j['completion'] < 1]
    counter = max_uploads - len(active_jobs)
    print('Will upload a max of'+str(counter)+' sequences')
    for ix, s in sequences.iterrows():
        if counter == 0:
            break
        if s['sent']:
            continue
        sequences['sent'][ix] = upload_sequence(s, session)
        counter -= s['sent']
    print('Uploading done')
    sequences.to_csv(sequence_records, index = False)
    
    #scheduler.enter(interval, 1, upload_new_sequences, (scheduler,))
    
def download_labelled_sequences(scheduler):
    print('Downloading sequences...')
    try:
        token = login()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return
    session = requests.Session()
    session.headers.update({"Authorization": 'Bearer '+token})
    sequences = pd.read_csv(sequence_records)
    try:
        jobs_res = session.get(server_url+'/api/job')
        jobs_res.raise_for_status()
        jobs = jobs_res.json()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        return
    print(jobs)
    complete_jobs = [j for j in jobs if j['completion'] == 1]
    print('Will download '+str(len(complete_jobs))+' sequences')
    for job in complete_jobs:
        dataset_name = job['dataset']['name']
        dataset_id = job['dataset']['_id']
        job_id = job['_id']
        try:
            labels_res = session.get(server_url+'/labels/'+dataset_id+'/labels/'+job_id+'_labels.csv')
            labels_res.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            continue
        open(sequence_labels+'/'+dataset_name+'.csv', 'wb').write(labels_res.content)
        sequences.loc[sequences['name'] == dataset_name, 'labeled'] = True
        try:
            dataset_delete_res = session.delete(server_url+'/api/dataset/'+dataset_id)
            dataset_delete_res.raise_for_status()
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            continue
    sequences.to_csv(sequence_records, index = False)
    print('Downloading done')
    
    #scheduler.enter(interval, 1, download_labelled_sequences, (scheduler,))
    

In [61]:
upload_new_sequences(None)

Uploading sequences...
Will upload 10 sequences
Uploading done


In [65]:
download_labelled_sequences(None)

[{'_id': '5d3f2d2732d832001751686a', 'type': 'Tracking', 'priority': 0, 'classes': ['Person'], 'creator': '5ceeba2fdfd12000174c1c0f', 'createdAt': '2019-07-29T17:30:15.175Z', 'numberOfTasks': 1, 'completion': 1, 'dataset': {'_id': '5d3f2abbdd892b00102a4454', 'name': '1_cam_1_3', 'updatedAt': '2019-07-29T17:19:57.553Z', 'size': {'RGB': 13, 'Depth': 0}, 'processing': False}, 'state': 'Completed', 'channel': 'RGB'}]
Will download 1 sequences


In [6]:
s = sched.scheduler(time.time, time.sleep)
s.enter(1, 1, upload_new_sequences, (s,))
s.run()

je suis la execucion


FileNotFoundError: [Errno 2] No such file or directory: 'media/nas/PeopleDetection/up_realsense_frames/Data_Seq/sequence_labeling_record.json'